In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("lab.ipynb")

/Users/ericzhao/miniforge3/envs/dsc80/lib/python3.12/site-packages/otter/export/__init__.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


# Lab 3 – Merging and Pivoting

## DSC 80, Winter 2026

### Due Date: Monday, January 26th at 11:59PM

## Instructions

Welcome to the third DSC 80 lab this quarter!

Much like in DSC 10, this Jupyter Notebook contains the statements of the problems and provides code and Markdown cells to display your answers to the problems. Unlike DSC 10, the notebook is *only* for displaying a readable version of your final answers. The coding will be done in an accompanying `lab.py` file that is imported into the current notebook, and **you will only submit that `lab.py` file**, not this notebook!

Some additional guidelines:
- **Unlike in DSC 10, labs will have both public tests and hidden tests.** The bulk of your grade will come from your scores on hidden tests, which you will only see on Gradescope after the assignment deadline.
- **Do not change the function names in the `lab.py` file!** The functions in the `lab.py` file are how your assignment is graded, and they are graded by their name. If you changed something you weren't supposed to, you can find the original code in the [course GitHub repository](https://github.com/dsc-courses/dsc80-2026-wi).
- Notebooks are nice for testing and experimenting with different implementations before designing your function in your `lab.py` file. You can write code here, but make sure that all of your real work is in the `lab.py` file, since that's all you're submitting.
- You are encouraged to write your own additional helper functions to solve the lab, as long as they also end up in `lab.py`.

**To ensure that all of the work you want to submit is in `lab.py`, we've included a script named `lab-validation.py` in the lab folder. You shouldn't edit it, but instead, you should call it from the command line (e.g. the Terminal) to test your work.** More details on its usage are given at the bottom of this notebook.

**Importing code from `lab.py`**:

* Below, we import the `.py` file that's contained in the same directory as this notebook.
* We use the `autoreload` notebook extension to make changes to our `lab.py` file immediately available in our notebook. Without this extension, we would need to restart the notebook kernel to see any changes to `lab.py` in the notebook.
    - `autoreload` is necessary because, upon import, `lab.py` is compiled to bytecode (in the directory `__pycache__`). Subsequent imports of `lab` merely import the existing compiled python.

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from lab import *

In [4]:
import os
import io
from pathlib import Path
import pandas as pd
import numpy as np
np.set_printoptions(legacy='1.21')

<div class="alert alert-block alert-danger" markdown="1">

**There are only two functions in this lab in which you may use a `for`-loop:**
- **`read_linkedin_surveys` in Question 1.**
- **`read_student_surveys` in Question 2.**
    
**You may lose points if you use a `for`-loop or `while`-loop in any other question!**

</div>

## Part 1: Combining Data

### Question 1 – Making Connections 🤝

A group of students decided to send out a survey to their connections on LinkedIn. Each student asks 1000 of their connections for their first and last name, the company they currently work at, their job title, their email, and the university they attended.

**Your job is to combine all the data contained in the files `survey*.csv` (stored within the `data/responses` folder) into a single DataFrame. The number of files and the number of rows in each file may vary, so don't hardcode your answers!** To do so, implement the following two functions.

#### `read_linkedin_surveys`

Complete the implementation of the function `read_linkedin_surveys`, which takes in a string or Path corresponding to the directory where the `survey*.csv` files are located. The function outputs a single DataFrame with six columns titled `'first name'`, `'last name'`, `'current company'`, `'job title'`, `'email'`, and `'university'` (in that order) containing all of the survey information from all of the `.csv` files. 

Make sure to reset the index of the combined DataFrame before returning it so that the index is unique. When the function takes in an invalid directory, it should raise a `FileNotFoundError`.

***Hints***:

- Take a look at a few of the files in the `responses` folder. You may have to clean up the column names to combine the DataFrames!

- You can iterate through the files in a directory using `.iterdir()`.

- Use `Path()` on the input. Calling `.iterdir()` on an invalid `Path()` object will automatically throw a `FileNotFoundError`.

<br>

#### `linkedin_stats`

Complete the implementation of the function `linkedin_stats`, which takes in a DataFrame returned by `read_linkedin_surveys` and returns a list containing, in the following order: 
- The proportion of people who went to a university with the string `'Ohio'` in its name that have the string `'Programmer'` somewhere in their job title.
- The number of distinct job titles that **end** with the exact string `'Engineer'`. Note that we're asking for the number of job titles, **not** the number of people!
- The job title that has the longest name (or any such job title in the case of a tie).
- The number of people who have the word `'manager'` in their job title, uppercase or lowercase (`'Manager'`, `'manager'`, and `'mANAgeR'` should all count).

In [9]:
def read_linkedin_surveys(survey_dir):
    p = Path(survey_dir)
    dfs = []

    for f in p.iterdir():
        if f.name.startswith("survey") and f.suffix == ".csv":
            df = pd.read_csv(f)

            df.columns = [
                "first name",
                "last name",
                "current company",
                "job title",
                "email",
                "university",
            ]

            dfs.append(df)
        
    return pd.concat(dfs, ignore_index=True)

In [10]:
def linkedin_stats(read_linkedin_surveys):
    ohio = read_linkedin_surveys["university"].str.contains("Ohio", na=False)
    prog = read_linkedin_surveys["job title"].str.contains("Programmer", na=False)
    prop = (ohio & prog).sum() / ohio.sum()

    engineer_titles = read_linkedin_surveys["job title"].str.endswith("Engineer", na=False)
    num_engineer_titles = read_linkedin_surveys.loc[engineer_titles, "job title"].nunique()

    longest_title = read_linkedin_surveys.loc[read_linkedin_surveys["job title"].str.len().idxmax(), "job title"]
    
    num_mnagers = read_linkedin_surveys["job title"].str.contains("manager", case=False, na=False).sum()
    return [prop, num_engineer_titles, longest_title, num_mnagers]

In [11]:
# do not edit this cell -- it is needed for the tests
dirname = Path('data') / 'responses'
q1_out = read_linkedin_surveys(dirname)
stats_out = linkedin_stats(q1_out)

In [12]:
grader.check("q1")

q1 results: All test cases passed!

### Question 2 – Survey Says... 👨‍👩‍👧‍👦

Professor Aritra often sends out extra credit surveys asking students for their favorite animals, movies, and other favorite things. These surveys are stored in the `data/extra-credit-surveys` folder. Each file in that folder corresponds to a different survey question (except for `favorite1.csv`, which contains students' names and IDs).

Here's how extra credit works:
- Each student who has completed at least 50% of the survey questions receives 5 points of extra credit.
- If there is at least one survey question that at least 90% of the class answered (e.g. favorite animal), **everyone** in the class receives 1 point of extra credit. This overall class extra credit only applies twice, so if for example 95% of students answer the favorite color survey question, 91% answer the favorite animal survey question, and 97% answer the favorite movie question, the entire class receives 2 extra points, not 3.
- Note that this means that the most extra credit any student can earn is 7 points.

#### `read_student_surveys`

Complete the implementation of the function `read_student_surveys` which takes in a string or Path corresponding to the directory where the `favorite*.csv` files are located. The function should output a DataFrame containing students' names and all of the survey data from all surveys, indexed by student ID (a value 1-1000). As in the previous problem, use `Path()` to build file paths so that `read_student_surveys` throws a `FileNotFoundError` when the input is an invalid directory.

<br>

#### `check_credit`

Complete the implementation of the function `check_credit` which takes in a DataFrame returned by `read_student_surveys` and outputs a DataFrame indexed by student ID (a value 1-1000) with two columns:
- `'name'`, containing the name of each student, and
- `'ec'`, containing the number of extra credit points each student earned.

***Note***: For the genres survey, `'(no genres listed)'` does not count as a valid response for receiving extra credit. 

In [30]:
def read_student_surveys(fav_dir):
    path = Path(fav_dir)
    if not path.exists():
        raise FileNotFoundError
    
    files = sorted(path.glob("favorite*.csv"))
    
    df = pd.read_csv(files[0]).set_index("id")

    for f in files[1:]:
        temp = pd.read_csv(f).set_index("id")
        df = df.join(temp)


    return df

In [31]:
def check_credit(read_student_surv):

    surveys = read_student_surv.drop(columns="name")

    surveys = surveys.replace("(no genres listed)", pd.NA)

    answered_frac = surveys.notna().mean(axis=1)
    student_ec = (answered_frac >= 0.5) * 5

    question_frac = surveys.notna().mean()
    class_ec = min(2, (question_frac >= 0.9).sum())

    return pd.DataFrame({
        "name": read_student_surv["name"],
        "ec": student_ec + class_ec
    })

In [32]:
# do not edit this cell -- it is needed for the tests
dirname = Path('data') / 'extra-credit-surveys'
q2_out = read_student_surveys(dirname)
check_credit_out = check_credit(q2_out)

In [33]:
grader.check("q2")

q2 results: All test cases passed!

### Question 3 – Paw Patrol 🐾

You are analyzing data from a veterinarian clinic in Michigan. The datasets contain several types of information from the clinic, including its customers (pet owners), pets, available procedures, and procedure history. The column names are self-explanatory. The following DataFrames are provided to you:
-  `owners` stores the customer information, where every `'OwnerID'` is unique.
-  `pets` stores the pet information, where each pet belongs to a customer in `owners`.
-  `procedure_detail` contains a catalog of procedures that are offered by the clinic.
-  `procedure_history` has procedure records for many animals, including but not limited to the ones in `pets`.
  
Complete the implementation of the following three functions, which each ask you to answer a specific question.

#### `most_popular_procedure_type`

What is the most popular `'ProcedureType'` amongst all pets in the `pets` DataFrame? Complete the implementation of the function `most_popular_procedure_type`, which takes in two DataFrames, `pets` and `procedure_history`, and returns the name of the most popular `'ProcedureType'` as a string.

Note that some pets in `pets` may not have had any procedures performed. Also, some pets that have had procedures done are not in the `pets` DataFrame.

<br>

#### `pet_name_by_owner`

What is the name of each customer's pet(s)? Complete the implementation of the function `pet_name_by_owner`, which takes in two DataFrames, `owners` and `pets`, and returns a Series whose index contains owner first names, and whose values are pet names as **strings**. If an owner has multiple pets, the value corresponding to that owner should instead be a **list of pet names as strings**.

Note that owner first names are not necessarily unique, and so the Series you return will not necessarily have a unique index.

<br>

#### `total_cost_per_city`

Note that the `owners` DataFrame has a `'City'` column, describing the city in which each pet owner and their pets live. The veterinary clinic is considering moving to a new location and wants to use the available data to determine which city brings in the most money. 

Complete the implementation of the function `total_cost_per_city`, which takes in four DataFrames, `owners`, `pets`, `procedure_history`, and `procedure_detail`, and returns a Series indexed by `'City'` that describes the total amount that pet owners in each city have spent on pets' procedures. The returned Series should include all cities that appear in `owners`, even if no pet owners from that city have spent any money on procedures.

***Hint***: At some point, you may have to merge on multiple columns.

In [38]:
def most_popular_procedure_type(pets, procedure_history):
    in_pets = procedure_history[procedure_history["PetID"].isin(pets["PetID"])]
    return in_pets["ProcedureType"].value_counts().idxmax()

In [ ]:
def pet_name_by_owner(owners, pets):
    first_col = "FirstName" if "FirstName" in owners.columns else "Name"

    pet_lists = pets.groupby("OwnerID")["Name"].apply(list)

    def convert(owner_id):
        xs = pet_lists.get(owner_id, [])
        if len(xs) == 1:
            return xs[0]
        return xs

    values = owners["OwnerID"].apply(convert)
    return pd.Series(values.values, index=owners[first_col])


In [40]:
def total_cost_per_city(owners, pets, procedure_history, procedure_detail):

    pet_city = pets.merge(
        owners[["OwnerID", "City"]],
        on="OwnerID",
        how="left"
    )[["PetID", "City"]]

    hist = procedure_history[procedure_history["PetID"].isin(pets["PetID"])]

    hist = hist.merge(
        procedure_detail[["ProcedureType", "ProcedureSubCode", "Price"]],
        on=["ProcedureType", "ProcedureSubCode"],
        how="left"
    )

    hist = hist.merge(pet_city, on="PetID", how="left")

    spend = hist.groupby("City")["Price"].sum().fillna(0)

    all_cities = owners["City"].drop_duplicates()
    return spend.reindex(all_cities, fill_value=0)

In [46]:
# do not edit this cell -- it is needed for the tests
owners_fp = Path('data') / 'pets' / 'Owners.csv'
pets_fp = Path('data') / 'pets' / 'Pets.csv'
procedure_detail_fp = Path('data') / 'pets' / 'ProceduresDetails.csv'
procedure_history_fp =  Path('data') / 'pets' / 'ProceduresHistory.csv'

owners = pd.read_csv(owners_fp)
pets = pd.read_csv(pets_fp)
procedure_detail = pd.read_csv(procedure_detail_fp)
procedure_history = pd.read_csv(procedure_history_fp)

out_01 = most_popular_procedure_type(pets, procedure_history)
out_02 = pet_name_by_owner(owners, pets)
out_03 = total_cost_per_city(owners, pets, procedure_history, procedure_detail)

In [47]:
grader.check("q3")

q3 results: All test cases passed!

## Part 2: Pivot Tables

### Question 4 – Stationery Store Sales ✏📕📏

Recall from [Lecture 4](https://dsc80.com/resources/lectures/lec04/lec04.html#Pivot-tables:-an-extension-of-grouping), a pivot table allows you to aggregate the entries in a DataFrame based on two categorical columns. In this question, you are given a simple dataset, `data/sales.csv`, representing sales at a stationery store, and you are asked to solve a few problems using the `pivot_table` method. 

**We have provided example outlines for the DataFrames you need to create, but yours may have a different number of rows and columns and different values.**

***Note***: If it helps, [here](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html) is another great resource that provides an overview of `pivot_table` with many examples from the Titanic dataset.


#### `average_seller`

Complete the implementation of the function `average_seller`, which takes in the `sales` DataFrame and returns a DataFrame that contains the average sales for each seller, indexed by `'Name'` and containing the column `'Average Sales'`. There should not be any `NaN`s.

***Note***: You can implement `average_seller` without using `pivot_table`, but try to use `pivot_table` for practice.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Average Sales</th>
    </tr>
    <tr>
      <th>Name</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Anthony</th>
      <td>0</td>
    </tr>
    <tr>
      <th>John</th>
      <td>0</td>
    </tr>
    <tr>
      <th>Jose</th>
      <td>0</td>
    </tr>
  </tbody>
</table>

<br>

#### `product_name`

Complete the implementation of the function `product_name` that takes in the `sales` DataFrame and returns a DataFrame that contains the total sales for each product, indexed by `'Name'`. Do not fill in `NaN`s.

<table border="1" class="dataframe">
  <thead>
    <tr>
      <th>Product</th>
      <th>book</th>
      <th>eraser</th>
      <th>pen</th>
    </tr>
    <tr>
      <th>Name</th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Anthony</th>
      <td>NaN</td>
      <td>0</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>John</th>
      <td>NaN</td>
      <td>0</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>Jose</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
  </tbody>
</table>

<br>

#### `count_product`

Complete the implementation of the function `count_product` that takes in the `sales` DataFrame and returns a DataFrame that contains the total number of items sold product-wise and name-wise per date. Replace `NaN`s with 0s. Don't reset the index after pivoting.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Date</th>
      <th>01.01.2022</th>
      <th>02.20.2023</th>
      <th>02.25.2025</th>
    </tr>
    <tr>
      <th>Product</th>
      <th>Name</th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th rowspan="3" valign="top">book</th>
      <th>Anthony</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>John</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>Jose</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>eraser</th>
      <th>Jose</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>pen</th>
      <th>Anthony</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
  </tbody>
</table>
<br>

#### `total_by_month`

Complete the implementation of the function `total_by_month` that takes in the `sales` DataFrame and returns a pivot table that contains the total sales name-wise and product-wise per month. Replace `NaN`s with 0s. Don't reset the index after pivoting.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Month</th>
      <th>February</th>
      <th>January</th>
      <th>July</th>
    </tr>
    <tr>
      <th>Name</th>
      <th>Product</th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th rowspan="4" valign="top">Anthony</th>
      <th>book</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>pen</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>ruler</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>stapler</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>John</th>
      <th>book</th>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
  </tbody>
</table>

In [68]:
def average_seller(sales):
    out = sales.pivot_table(
        index="Name",
        values="Total",
        aggfunc="mean"
    )
    out = out.rename(columns={"Total": "Average Sales"})
    return out.fillna(0)

In [69]:
def product_name(sales):
    return sales.pivot_table(
        index="Name",
        columns="Product",
        values="Total",
        aggfunc="sum"
    )

In [70]:
def count_product(sales):
    return sales.pivot_table(
        index=["Product", "Name"],
        columns="Date",
        values="Total",
        aggfunc="size",
        fill_value=0
    )

In [74]:
def total_by_month(sales):
    sales = sales.copy()
    sales["Month"] = pd.to_datetime(sales["Date"]).dt.month

    return sales.pivot_table(
        index=["Name", "Product"],
        columns="Month",
        values="Total",
        aggfunc="sum",
        fill_value=0
    )

In [64]:
sales.columns


Index(['Name', 'Product', 'Date', 'Total'], dtype='object')

In [75]:
# don't change this cell -- it is needed for the tests to work
fp = Path('data') / 'sales.csv'
sales = pd.read_csv(fp)
q4_average_seller_out = average_seller(sales)
q4_product_name_out = product_name(sales)
q4_count_product_out = count_product(sales)
q4_total_by_month_out = total_by_month(sales)

In [76]:
grader.check("q4")

q4 results: All test cases passed!

## Congratulations! You're done with Lab 3! 🏁

As a reminder, all of the work you want to submit needs to be in `lab.py`.

To ensure that all of the work you want to submit is in `lab.py`, we've included a script named `lab-validation.py` in the lab folder. You shouldn't edit it, but instead, you should call it from the command line (e.g. the Terminal) to test your work.

Once you've finished the lab, you should open the command line and run, in the directory for this lab:

```
python lab-validation.py
```

**This will run all of the `grader.check` cells that you see in this notebook, but only using the code in `lab.py` – that is, it doesn't look at any of the code in this notebook. If all of your `grader.check` cells pass in this notebook but not all of them pass in your command line with the above command, then you likely have code in your notebook that isn't in your `lab.py`!**

You can also use `lab-validation.py` to test individual questions. For instance,

```
python lab-validation.py q1 q2 q4
```

will run the `grader.check` cells for Questions 1, 2, and 4 – again, only using the code in `lab.py`. [This video](https://www.loom.com/share/0ea254b85b2745e59322b5e5a8692e91?sid=5acc92e6-0dfe-4555-9b6a-8115b6a52f99) how to use the script as well.

Once `python lab-validation.py` shows that you're passing all test cases, you're ready to submit your `lab.py` (and only your `lab.py`) to Gradescope. Once submitting to Gradescope, make sure to stick around until all test cases pass.

There is also a call to `grader.check_all()` below in _this_ notebook, but make sure to also follow the steps above.

<div class="alert alert-block alert-danger" markdown="1">

**There are only two functions in this lab in which you may use a `for`-loop:**
- **`read_linkedin_surveys` in Question 1.**
- **`read_student_surveys` in Question 2.**
    
**You may lose points if you use a `for`-loop or `while`-loop in any other question!**

</div>

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [77]:
grader.check_all()

q1 results: All test cases passed!

q2 results: All test cases passed!

q3 results: All test cases passed!

q4 results: All test cases passed!